## Retrieve Weather Data

In [17]:
# Import libraries.
import pandas as pd
import numpy as np
from citipy import citipy
import requests
from config import weather_api_key

In [18]:
# Select US cities.
lats = np.random.uniform(low=20, high=70, size=7000)
lngs = np.random.uniform(low=-160, high=-65, size=7000)
lat_lng = zip(lats, lngs)

In [19]:
coordinates = list(lat_lng)

In [20]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [21]:
# Create a list for holding the cities.
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count.
len(cities)

1183

In [22]:
# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes.
    if(i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ", "+")
    
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()

        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_weather_des = city_weather["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]

        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                         "Lat": city_lat,
                         "Lng": city_lng,
                         "Max Temp": city_max_temp,
                         "Humidity": city_humidity,
                         "Cloudiness": city_clouds,
                         "Wind Speed": city_wind,
                         "Current Description": city_weather_des,
                         "Country": city_country})
        
        # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | fort nelson
Processing Record 2 of Set 1 | bethel
Processing Record 3 of Set 1 | north bend
Processing Record 4 of Set 1 | lompoc
Processing Record 5 of Set 1 | wabash
Processing Record 6 of Set 1 | kapaa
Processing Record 7 of Set 1 | santa barbara
Processing Record 8 of Set 1 | hastings
Processing Record 9 of Set 1 | havre
Processing Record 10 of Set 1 | duluth
Processing Record 11 of Set 1 | clyde river
Processing Record 12 of Set 1 | teacapan
Processing Record 13 of Set 1 | billings
Processing Record 14 of Set 1 | iqaluit
Processing Record 15 of Set 1 | fortuna
Processing Record 16 of Set 1 | thompson
Processing Record 17 of Set 1 | cockburn town
Processing Record 18 of Set 1 | kodiak
Processing Record 19 of Set 1 | kahului
Processing Record 20 of Set 1 | edson
Processing Record 21 of Set 1 | starkville
Processing Record 22 of Set 1 | attawapiskat
City not found. Skipping...
Processing Record

In [23]:
city_df = pd.DataFrame(city_data)
city_df.shape

(1163, 9)

In [24]:
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", 
                    "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_df = city_df[new_column_order]
city_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Fort Nelson,CA,58.8053,-122.7002,62.29,41,22,1.88,few clouds
1,Bethel,US,41.3712,-73.4140,66.56,83,5,3.53,clear sky
2,North Bend,US,43.4065,-124.2243,61.92,100,100,1.97,overcast clouds
3,Lompoc,US,34.6391,-120.4579,73.56,69,75,12.66,broken clouds
4,Wabash,US,40.7978,-85.8205,73.87,79,2,7.76,clear sky


In [25]:
new_df = city_df[city_df.Country == 'US']
new_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,Bethel,US,41.3712,-73.4140,66.56,83,5,3.53,clear sky
2,North Bend,US,43.4065,-124.2243,61.92,100,100,1.97,overcast clouds
3,Lompoc,US,34.6391,-120.4579,73.56,69,75,12.66,broken clouds
4,Wabash,US,40.7978,-85.8205,73.87,79,2,7.76,clear sky
5,Kapaa,US,22.0752,-159.3190,83.19,69,61,5.01,broken clouds


In [28]:
new_df.shape

(673, 9)

In [29]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"

# Export the City_Data into a CSV.
new_df.to_csv(output_data_file, index_label="City_ID")